In [13]:
import scipy.stats as stats
import random
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_context("notebook", font_scale=1.5)
%matplotlib inline

/Users/Jake/anaconda/envs/snakes/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [123]:
current_q = np.array([1,0,0])
U = U_real
grad_U = U_grad_real
X = carX
Y = carY
hyperparams = car_hyperparams


In [172]:
    q = current_q
    p = np.random.normal(0,1,q.size) # independent statndard normal variates
    current_p = p
    q_list=q
    p_list=p
    


In [121]:
    # Half step for momentum at the beginning
    p = p - .25*grad_U(q,X,Y,hyperparams)/2

In [169]:
phi = q[0]
beta = q[1:3].reshape((2,1))
a = hyperparams[0]
b = hyperparams[1]
tau = hyperparams[2]   
b_p = beta.shape[0]
phi_part = (-n/2 - a + 1)/phi + b + 1/2*(beta.T @ (X.T @ X) @ beta - 2*Y.T @ X @ beta + Y.T @ Y)
beta_part = beta.T @ (phi*X.T @ X + tau*np.eye(b_p)) - phi*Y.T @ X
res = np.append(phi_part,beta_part)

In [179]:
res

array([ 62427.5,  -2149. , -38482. ])

In [132]:
beta.T @ (phi*X.T @ X + tau*np.eye(p)) - phi*Y.T @ X

-24.0

In [138]:
grad_U(q,X,Y,hyperparams)/2

array([ -1.20000000e+01,  -1.07450000e+03,  -1.92410000e+04])

In [ ]:
    # Make a full step for the position
    q = q + stepsize*p
    q_list = np.vstack((q_list,q))

In [ ]:
p = p - stepsize*grad_U(q,X,Y,hyperparams)
p_list = np.vstack((p_list,p))

In [91]:
def leapfrog_real(U, grad_U, current_q, X, Y, hyperparams, stepsize=0.25, steps=25):
    '''
    Performs the leapfrog algorithm for HMC, with 1 accept/reject step
    '''
    a = hyperparams[0]
    b = hyperparams[1]
    tau = hyperparams[2]
    
    q = current_q
    p = np.random.normal(0,1,q.size) # independent statndard normal variates
    current_p = p
    q_list=q
    p_list=p
    
    # Half step for momentum at the beginning
    p = p - stepsize*grad_U(q,X,Y,hyperparams)/2
    
    # Alternate full steps for position and momentum
    for i in range(steps):
        
        # Make a full step for the position
        q = q + stepsize*p
        q_list = np.vstack((q_list,q))
        
        # Make a full step for the momentum, except at end of trajectory
        if (i!=steps-1):
            p = p - stepsize*grad_U(q,X,Y,hyperparams)
            p_list = np.vstack((p_list,p))
    
    # Make a half step for momentum at the end
    p = p - stepsize*grad_U(q,X,Y,hyperparams)/2
    p_list = np.vstack((p_list,p))
    
    # Negate momentum at end of trajectory to make the proposal symmetric
    p = -p
    
    # Evaluate potential and kinetic energies at start and end of trajectory
    current_U = U(current_q,X,Y,hyperparams)
    current_K = sum(current_p**2)/2
    proposed_U = U(q,X,Y,hyperparams)
    proposed_K = sum(p**2)/2

    # Accept or reject the state at end of trajectory, returning either the postion at the end
    # of the trajectory or the initial position
    if(q[0] <= 0):
        print("Proposed Phi <= 0")
        return(current_q,p_list,q_list) # Reject
    elif (np.random.uniform(0,1) < np.exp(current_U-proposed_U+current_K-proposed_K)):
        
        return(q,p_list,q_list) # Accept
    else:
        return(current_q,p_list,q_list) # Reject
    

In [108]:
def HMC_real(U, grad_U, current_q, hyperparams, stepsize=0.2, steps=25, max_iter= 20, step_range=0):
    '''
    Performs max_iter iterations of HMC with leapfrog discritization method
    To randomize stepsize, set step_range to be nonzero
    '''
    
    current_q_list = current_q
    
    for i in range(max_iter):
        
        #Randomize stepsize if called for
        #Note if step_range = 0, it just draws stepsize
        stepsize_r = np.random.uniform(stepsize-step_range,stepsize+step_range)

        #Keep the position variable after every itertion
        current_q,p_list,q_list = leapfrog_real(U, grad_U, current_q, Sigma, stepsize_r, steps)
        current_q_list = np.vstack((current_q_list,current_q))
    return(current_q_list[:-1,:])

In [186]:
def U_real(theta,X,Y,hyperparams):
    phi = theta[0]
    beta = theta[1:3].reshape((2,1))
    a = hyperparams[0]
    b = hyperparams[1]
    tau = hyperparams[2]
    
    p = beta.shape[0]
    return((-n/2 - a + 1)*np.log(phi) + phi*b + 1/2*(beta.T @ (phi*X.T @ X + tau*np.eye(p)) @ beta - 2*phi*Y.T @ X @ beta ))

def U_grad_real(theta,X,Y,hyperparams):
    phi = theta[0]
    beta = theta[1:3].reshape((2,1))
    a = hyperparams[0]
    b = hyperparams[1]
    tau = hyperparams[2]
    
    b_p = beta.shape[0]
    phi_part = (-n/2 - a + 1)/phi + b + 1/2*(beta.T @ (X.T @ X) @ beta - 2*Y.T @ X @ beta + Y.T @ Y)
    beta_part = phi*beta.T @ (phi*X.T @ X + tau*np.eye(b_p)) - phi*Y.T @ X
    return(np.append(phi_part,beta_part))

In [188]:
def U_real_flat(theta,X,Y,hyperparams):
    phi = theta[0]
    beta = theta[1:3].reshape((2,1))
    a = hyperparams[0]
    b = hyperparams[1]
    tau = hyperparams[2]
    
    p = beta.shape[0]
    return((-n/2 - a + 1)*np.log(phi)+ 1/2*(beta.T @ (phi*X.T @ X) @ beta - 2*phi*Y.T @ X @ beta ))

def U_grad_real_flat(theta,X,Y,hyperparams):
    phi = theta[0]
    beta = theta[1:3].reshape((2,1))
    a = hyperparams[0]
    b = hyperparams[1]
    tau = hyperparams[2]
    
    b_p = beta.shape[0]
    phi_part = -n/(2*phi) + 1/2*(beta.T @ (X.T @ X) @ beta - 2*Y.T @ X @ beta + Y.T @ Y)
    beta_part = phi*beta.T @ (phi*X.T @ X) - phi*Y.T @ X
    return(np.append(phi_part,beta_part))

In [161]:
cars = pd.read_table('cars.txt',  delimiter=' ')
n = cars.shape[0]
carX = np.vstack((np.repeat(1,n),np.array(cars.ix[:,0]))).T
carY = np.array(cars.ix[:,1]).reshape(n,1)


In [197]:
#Testing
current_q = np.array([1,0,0])
car_hyperparams = np.array([.1,.1,.1])


q,p_list,q_list=leapfrog_real(U_real_flat, U_grad_real_flat, current_q,carX,carY, car_hyperparams, stepsize=0.01, steps=20)

In [198]:
q_list

array([[  1.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [ -2.11778959e+000,   1.18855074e-001,   1.92088147e+000],
       [ -6.52247075e+000,  -8.83439834e-001,  -1.57451694e+001],
       [ -2.42994022e+002,   4.84782495e+001,   8.30448415e+002],
       [ -4.56516273e+005,  -3.78994130e+006,  -6.50828158e+007],
       [ -2.82056919e+015,   1.04835580e+018,   1.80029092e+019],
       [ -2.15818878e+038,  -1.10699599e+049,  -1.90099089e+050],
       [ -2.40637951e+100,   6.84366496e+125,   1.17522962e+127],
       [ -9.19707026e+253,              -inf,              -inf],
       [             -inf,               nan,               nan],
       [              nan,               nan,               nan],
       [              nan,               nan,               nan],
       [              nan,               nan,               nan],
       [              nan,               nan,               nan],
       [              nan,               nan,               nan],
       [  

In [107]:
hmc_output_real=HMC(U_real, U_grad_real, current_q, carX,carY, car_hyperparams, stepsize=0.01, steps=20, max_iter=200)

array([[  1.        ,   0.        ,   0.        ],
       [  1.0047968 ,   0.21711022,   3.84472393],
       [  1.01198209,   0.56752536,  10.29573343],
       [  1.02153884,   0.54774258,  10.70865328],
       [  1.03344478,   0.1218875 ,   4.47003998],
       [  1.04767273,  -0.21612859,   0.06476033],
       [  1.06419101,  -0.10794488,   3.65047486],
       [  1.08296393,   0.15907378,  10.2966211 ],
       [  1.10395222,   0.03207045,  10.51395324],
       [  1.12711358,  -0.514363  ,   3.87130199],
       [  1.15240314,  -0.90944075,   0.17606777],
       [  1.17977398,  -0.8195897 ,   5.16248904],
       [  1.20917758,  -0.68680285,  11.24671849],
       [  1.24056428,  -1.07729683,   8.71196832],
       [  1.27388364,  -1.76010049,   1.53141649],
       [  1.30908486,  -2.03437572,   1.74723435],
       [  1.34611705,  -1.90879049,   9.21772817],
       [  1.38492958,  -2.14720465,  10.83274416],
       [  1.42547227,  -2.93068523,   3.49017767],
       [  1.46769563,  -3.46366